<a href="https://colab.research.google.com/github/tenoriolms/databank_CH4/blob/main/_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.python.org/pt-br/3/tutorial/modules.html

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import missingno as msno

In [ ]:
'''
import sklearn.datasets
import sklearn.model_selection
import sklearn.metrics
import sklearn.ensemble #bibliotecas de aprendizado de máquina
'''

#Declaração de Variáveis

In [ ]:
#ID do databank
data = '962'

#dict para armazenar dados da função ID
ID_dict = {}

#gases com permeabilidades presentes no banco de dados ordem crescente de kinetic diameter
gases = ('He','H2','CO2','O2','H2S','N2','CO','CH4','C2H6','SF6') #EM ORDEM
gases_kinetic_diameter = {'He':260,
                          'H2':289,
                          'CO2':330,
                          'Ar':340,
                          'O2':346,
                          'H2S':360,
                          'N2':364,
                          'CO':376,
                          'CH4':380,
                          'C2H6':444.3,
                          'SF6':550,
                          'none':0}
gases_effec_diameter = {'He':178, #T>Tg
                        'H2':214,
                        'CO2':302,
                        'O2':289,
                        'Ar':297,
                        'H2S':None,
                        'N2':304,
                        'CO':304,
                        'CH4':318,
                        'C2H6':346,
                        'SF6':None,
                        'none':0}
                        
                        
#frequencia de dados para cada gas em situação "pura" e "mixtura"
count_pure = {}
count_mixture = {}


#nome das colunas correspondentes à características da membrana e do processo seus respectivos índices na coluna
#EM ORDEM:
columns_membrane = ['type', 'description', 'support_material', 'subtype', 'filler_loading',
                    'previous_thickness', 'mean_thickness', 'mean_pore_size', 'pore_volume',
                    'specific_surface_area',  'aging']
columns_process = ['surface_area', 'temperature', 'feed_pressure', 'permeate_pressure',
                   'delta_pressure', 'feed_flow_rate', 'sweep_gas', 'sweep_gas_flow',
                   'stage_cut']
columns_others = ['provided_data_type', 'in_reference_data_location', 'reference', 'url']

columns_membrane_index = {}
columns_process_index = {}
columns_others_index = {}

#A performance de cada gás foi representada pela seguintes variáveis:
prefix1='x_' #Fração mássica/molar/volumétrica
prefix2='Py_' #Permeabilidade
prefix3='Pe_' #Permeância

##GitHub info

In [ ]:
git_username = 'tenoriolms' #username no GitHub
git_repository = 'databank_CH4' #Nome do repositório

git_token = 'não pode' #token para acesso do repositório.
#O github possui um algoritmo para verificar se dentro de cada arquivo importado existe
#o token de acesso criado. Caso existir, esse token é revogado.
#Como esse notebook irá ser exportado para o github. O token não pode ser escrito aqui.

!git config --global user.email "lhucas_tenorio@hotmail.com"
!git config --global user.name "tenoriolms"

#Funções

##ID(index)

In [ ]:
#ID = string utilizada para identificar uma membrana e suas circunstâncias de utilização no banco de dados
#A ID é a soma das variáveis (em forma de strings) que podem variar em uma dada referência
def ID(i, df):
  if i in ID_dict:
    return ID_dict[i]
  else:
    
    aux = str(df['description'][i]) + str(df['filler_loading'][i]) \
    + str(df['mean_thickness'][i]) + str(df['mean_pore_size'][i]) \
    + str(df['temperature'][i]) + str(df['feed_pressure'][i]) \
    + str(df['delta_pressure'][i]) + str(df['feed_flow_rate'][i]) \
    + str(df['stage_cut'][i]) + str(df['aging'][i]) \
    + str(df['reference'][i])
    ID_dict[i] = aux
    
    return aux

##submit_file(git_export_file)

In [ ]:
#exportar para o GitHub
def submit_file(git_export_file):
  !git clone https://{git_token}@github.com/{git_username}/{git_repository}
  !cp {git_export_file} {git_repository}
  %cd {git_repository}
  !git add {git_export_file}
  !git commit -m 'Add/Atualizar arquivo {input_file}'
  !git push -u origin
  %cd ..
  !rm -rf {git_repository}

##import_file(git_import_file)

In [ ]:
#importar um arquivo do GitHub
def import_file(git_import_file):
  !git clone https://{git_token}@github.com/{git_username}/{git_repository}
  !cp {git_repository}/{git_import_file} .
  !rm -rf {git_repository}

##Gráficos

### plt_hist_by(df,variable,by)

In [ ]:
def plt_hist_by(df = pd.DataFrame(),
                variable = [],
                by = '',
                consider_none = False,
                consider_zeros = False):
  #https://plotly.com/python/histograms/
  import plotly.graph_objects as go

  fig = go.Figure()

  #Filtrar apenas as linha que possuem dados de "variavel"
  for i in variable:
    
    if (consider_none==False):
      df.loc[ df[i]=='none', i ] = np.nan
      df.loc[ df[i]=='None', i ] = np.nan

    if (consider_zeros==False):
      df.loc[ df[i]==0, i ] = np.nan

    df_aux = df.loc[ df[i].notna(), by]

    fig.add_trace(go.Histogram(
        x=df_aux,
        histnorm='',
        name=i, # name used in legend and hover labels
        #marker_color='#EB89B5',
        #opacity=0.75
        ))

  fig.update_layout(
      title_text=f'Quantity of data by each {by}', # title of plot
      xaxis_title_text=by, # xaxis label
      yaxis_title_text='Count', # yaxis label
      bargap=0.2, # gap between bars of adjacent location coordinates
      bargroupgap=0.1 # gap between bars of the same location coordinates
      )

  fig.show()

### plt_hist_by(df,variable,by)

In [ ]:
def plt_hist_columns(df):
  aux = []
  for i in df.columns:
    aux += [i]
    if (len(aux)==4):
      try:
        df[aux] = df[aux].astype(float)
      except:
        print()
      df[aux].hist()
      aux = []
  df[aux].hist()